<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Wavenet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

from keras.layers import Conv1D
from keras.utils.conv_utils import conv_output_length
from keras import layers

# Classe CaudalDilatedConv1D

In [ ]:
class CausalDilatedConv1D(Conv1D):
    def __init__(self, nb_filter, filter_length, init='glorot_uniform', activation=None, weights=None,
                 border_mode='valid', subsample_length=1, atrous_rate=1, W_regularizer=None, b_regularizer=None,
                 activity_regularizer=None, W_constraint=None, b_constraint=None, bias=True, causal=False, **kwargs):
        super(CausalDilatedConv1D, self).__init__(nb_filter, filter_length, weights=weights, activation=activation, 
                padding=border_mode, strides=subsample_length, dilation_rate=atrous_rate, kernel_regularizer=W_regularizer, 
                bias_regularizer=b_regularizer, activity_regularizer=activity_regularizer, kernel_constraint=W_constraint, 
                bias_constraint=b_constraint, use_bias=bias, **kwargs)
        self.causal = causal
        self.nb_filter = nb_filter
        self.atrous_rate = atrous_rate
        self.filter_length = filter_length
        self.subsample_length = subsample_length
        self.border_mode = border_mode
        if self.causal and border_mode != 'valid':
            raise ValueError("Causal mode dictates border_mode=valid.")

    def compute_output_shape(self, input_shape):
        input_length = input_shape[1]
        if self.causal:
            input_length += self.atrous_rate * (self.filter_length - 1)
        length = conv_output_length(input_length, self.filter_length, self.border_mode, self.strides[0], dilation=self.atrous_rate)
        return (input_shape[0], length, self.nb_filter)

    def call(self, x, mask=None):
        if self.causal:
            x = K.temporal_padding(x, padding=(self.atrous_rate * (self.filter_length - 1), 0))
        # return super(CausalAtrousConvolution1D, self).call(x, mask)
        return super(CausalDilatedConv1D, self).call(x)

# Wavenet Model

**1. Méthodes**

**2. Modèle**

In [ ]:
def build_model(fragment_length, nb_filters, nb_output_bins, dilation_depth, nb_stacks, use_skip_connections,learn_all_outputs, desired_sample_rate, use_bias, res_l2, final_l2):
  def residual_block(x,i,s):
    original_x = x
    
    tanh_out = (nb_filters, 2, dilation_rate=2 ** i, padding='valid', causal=True,
                                         use_bias=use_bias, name='dilated_conv_%d_tanh_s%d' % (2 ** i, s), activation='tanh',
                                         kernel_regularizer=tf.keras.regularizers.l2(res_l2))(x)
    sigm_out = (nb_filters, 2, dilation_rate=2 ** i, padding='valid', causal=True,
                                         use_bias=use_bias,
                                         name='dilated_conv_%d_sigm_s%d' % (2 ** i, s), activation='sigmoid',
                                         kernel_regularizer=tf.keras.regularizers.l2(res_l2))(x)
    x = layers.Multiply(name='gated_activation_%d_s%d' % (i, s))([tanh_out, sigm_out])

    res_x = layers.Convolution1D(nb_filters, 1, padding='same', use_bias=use_bias, kernel_regularizer=tf.keras.regularizers.l2(res_l2))(x)
    skip_x = layers.Convolution1D(nb_filters, 1, padding='same', use_bias=use_bias, kernel_regularizer=tf.keras.regularizers.l2(res_l2))(x)
    res_x = layers.Add()([original_x, res_x])
    return res_x, skip_x
  
  input = tf.keras.layers.Input(shape=(fragment_length, nb_output_bins), name='input_part')
  out = input
  skip_connections = []
  out = CausalAtrousConvolution1D(nb_filters,2,dilation_rate=1,padding='valid',causal=True,name='initial_causal_conv')(out)
  
  for s in range(nb_stacks):
    for i in range(0, dilation_depth + 1):
      out, skip_out = residual_block(out)
      skip_connections.append(skip_out)
    
    if use_skip_connections:
      out = layers.Add()(skip_connections)
    
    out = layers.Activation('relu')(out)
    out = layers.Convolution1D(nb_output_bins, 1, padding='same',kernel_regularizer=tf.keras.kernel_regularizer.l2(final_l2))(out)
    out = layers.Activation('relu')(out)
    out = layers.Convolution1D(nb_output_bins, 1, padding='same')(out)

    if not learn_all_outputs:
        raise DeprecationWarning('Learning on just all outputs is wasteful, now learning only inside receptive field.')
        out = layers.Lambda(lambda x: x[:, -1, :], output_shape=(out._keras_shape[-1],))(out)

    out = layers.Activation('softmax', name="output_softmax")(out)
    model = Model(input, out)

    receptive_field, receptive_field_ms = compute_receptive_field()
    return model

# Use the model

In [ ]:
def compute_receptive_field_(dilation_depth, nb_stacks):
    receptive_field = nb_stacks * (2 ** dilation_depth * 2) - (nb_stacks - 1)
    return receptive_field

In [ ]:
nb_filters = 256
nb_output_bins = 1
dilation_depth = 8
nb_stacks = 1
use_skip_connections = True
learn_all_outputs = True
use_bias = False
res_l2 = 0
final_l2 = 0

fragment_length = compute_receptive_field_(dilation_depth, nb_stacks)

In [ ]:
model = build_model(1024, nb_filters, nb_output_bins, dilation_depth, nb_stacks, use_skip_connections,learn_all_outputs, desired_sample_rate, use_bias, res_l2, final_l2)

ValueError: ignored